In [1]:
!pip install soynlp
!pip install keras-crf
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 43.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 38.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 2.1 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=acc5a1c66c166f2c9ba448461b27bcc43b878e00f9dbedf21a5660d9a17b6b7b
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [2]:
import pandas as pd
import re
import ast

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer

In [4]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, LSTM, Input, Bidirectional, TimeDistributed, Embedding, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras_crf import CRFModel
from seqeval.metrics import f1_score, classification_report

In [47]:
path = '/content/drive/MyDrive/Colab Notebooks/AURA2/data/'
import ast
train_df = pd.read_csv(path +'ner_train_df.csv')
sentences = []
ner_tags = []
for i in range(len(train_df)):
  if i % 10000 == 0:
    print("진행: ",i)
  sentences.append(ast.literal_eval(train_df['sentence'].iloc[i]))
  ner_tags.append(ast.literal_eval(train_df['ner_tags'].iloc[i]))

# 모든 단어를 사용하며 인덱스 1에는 단어 'OOV'를 할당.
src_tokenizer = Tokenizer(oov_token='OOV')
# 태깅 정보들은 내부적으로 대문자를 유지한 채 저장
tar_tokenizer = Tokenizer(lower=False)

src_tokenizer.fit_on_texts(sentences)
tar_tokenizer.fit_on_texts(ner_tags)

vocab_size = len(src_tokenizer.word_index) + 1
tag_size = len(tar_tokenizer.word_index) + 1
print('단어 집합의 크기 : {}'.format(vocab_size))
print('개체명 태깅 정보 집합의 크기 : {}'.format(tag_size))

word_to_index = src_tokenizer.word_index
index_to_word = src_tokenizer.index_word
ner_to_index = tar_tokenizer.word_index
index_to_ner = tar_tokenizer.index_word

index_to_ner[0] = 'PAD'

진행:  0
진행:  10000
진행:  20000
진행:  30000
진행:  40000
진행:  50000
진행:  60000
진행:  70000
진행:  80000
진행:  90000
진행:  100000
진행:  110000
진행:  120000
진행:  130000
진행:  140000
진행:  150000
진행:  160000
진행:  170000
진행:  180000
진행:  190000
진행:  200000
진행:  210000
진행:  220000
진행:  230000
진행:  240000
진행:  250000
진행:  260000
진행:  270000
진행:  280000
진행:  290000
진행:  300000
진행:  310000
진행:  320000
진행:  330000
진행:  340000
진행:  350000
단어 집합의 크기 : 232363
개체명 태깅 정보 집합의 크기 : 300


In [6]:
info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/Speaker_info.csv')
talk1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_10000.csv')
talk2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_20000.csv')
talk3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_30000.csv')
talk4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_40000.csv')
talk5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_50000.csv')
talk6 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_60000.csv')
talk7 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_70000.csv')
talk8 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_74665.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,4

In [8]:
talkid_sentence = dict([])
talks = [talk1, talk2, talk3, talk4, talk5, talk6, talk7, talk8]
for talk in talks :
  max_column = int(talk.columns[-1]) 
  for i in range(len(talk)) :
    sentences = []
    for j in range(1, max_column + 1) :
      if type(talk.loc[i, str(j)]) != str :
        break
      sentence = talk.loc[i, str(j)]
      sentence = re.sub('name[0-9]+', '', sentence) # name 삭제
      sentence = re.sub('[ㄱ-ㅎ]', '', sentence) # ㅋㅋㅋ ㅎㅎㅎ 삭제
      sentence=  re.sub('account', '', sentence) # account 삭제
      sentence=  re.sub('others', '', sentence) # others 삭제
      sentence = re.sub('[.]', '', sentence) # .삭제
      sentence = re.sub('[,]', '', sentence) # ,삭제
      sentences.append(sentence)
    talkid_sentence[talk.loc[i, 'talk_id']] = sentences

In [9]:
talkid_sentence

{'MDRW2100000001.1': [' 안녕하세요',
  ' ️',
  '  이거 해봐요><',
  ' 나의 직장인 멘탈 성향은  [안챙겨도 잘커요 탕비실 선인장] 당신의 멘탈 성향은 어떠신가요? 지금 테스트해보세요!  아앗',
  ' 오  안챙겨도 잘커요 너무 맞는데요ㅜㅜ? 자세한 내용은 더 알아가야겟지만~~',
  '  아녜여 챙겨주세요',
  ' 당연하쥬 누굴 챙길 여유는 저도 없는거같지만 그러나 점심 메뉴는 언제든 권한을 드릴게여 !!!!',
  ' 그렇게 큰 권한을 주신다구요?? 님 완전 대인배'],
 'MDRW2100000001.2': [' 목요일 점심메뉴도 생각해오세요 크크 전 닭가슴살 먹을거지만',
  ' 흠 그럼 저도 한번 도시락을 싸올까요',
  '  도시락쌀 기운이 있다싶으면 좋아요~~ 부지런한편이에요?',
  ' 저요? 저 완전 발발이에요',
  ' mbti p는 게으르잖아요 부지런한 p인건가 ',
  ' 맞아요 그래서 일을 벌려놔요 일을 저지르면 해결해야해서',
  '  오 멋있다 그래도',
  ' 여기 면접본것도 사실 일단 저질러서',
  '  근데 저도'],
 'MDRW2100000001.3': [' 전 용두사미 스타일~~ 일단 호기롭게 시작하고 끝이 미미해 그냥 해치우자! 느낌 ',
  ' 그래도 시작이 반이잖아요?!',
  ' 저도 그렇게 생각해요 뭐라도 해보는게 낫지!! ',
  ' 맞아요! 가만히 있는것보다야 훨 낫죠',
  ' 맞아요 ! 뭐라도 얻는게 있겠죠 헿 내일 잘쉬어요 ㅜㅜ 목요일에 엉덩이\r\n지켜야지',
  ' 님은 재밌게 다녀오세요 제 엉덩이',
  ' ',
  ' 잘 말려서 올게요'],
 'MDRW2100000001.4': [' 서서일하는것도 추천  ',
  ' 앟 그럼 너무 튀어요 안그래도 팀장님이 쳐다보는거 느껴진단말이에여',
  ' 아까 심리테스트에 관종이라고 적혀있던데?  관심 크게 한번 받으세여~!',
  '  아녜요 교수님이랑 친구들이 직장에선 자제하라고 했어여',
  ' 아까 채팅에서 이미 조금

In [10]:
sentences = []
for i in talkid_sentence :
  tmp = talkid_sentence[i]
  sentences += tmp

In [11]:
noun_extractor = LRNounExtractor_v2(verbose=True)
nouns = noun_extractor.train_extract(sentences)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 1840961 from 1675512 sents. mem=9.579 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=8516793, mem=13.298 Gb
[Noun Extractor] batch prediction was completed for 354782 words
[Noun Extractor] checked compounds. discovered 342464 compounds
[Noun Extractor] postprocessing detaching_features : 322122 -> 244068
[Noun Extractor] postprocessing ignore_features : 244068 -> 243594
[Noun Extractor] postprocessing ignore_NJ : 243594 -> 238685
[Noun Extractor] 238685 nouns (342464 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=14.400 Gb                    
[Noun Extractor] 61.66 % eojeols are covered


In [12]:
nounscore1 = dict([])
nounscore2 = dict([]) # ner모델 학습시 사용한 개체명을 nounscore에 1.0으로 포함

In [13]:
for i in nouns :
  nounscore1[i] = nouns[i].score

In [14]:
tokenizer = LTokenizer(scores=nounscore1)
talkid_tokens = dict([])
for i in talkid_sentence :
  sentences = talkid_sentence[i]
  tmp = []
  for j in sentences :
    tmp.append(tokenizer.tokenize(j))
  talkid_tokens[i] = tmp

In [15]:
embedding_dim = 64
hidden_units = 32
dropout_ratio = 0.3
max_len = 40

sequence_input = Input(shape=(max_len,),dtype=tf.int32, name='sequence_input')

model_embedding = Embedding(input_dim=vocab_size,
                              output_dim=embedding_dim,
                              input_length=max_len)(sequence_input)

model_bilstm = Bidirectional(LSTM(units=hidden_units, return_sequences=True))(model_embedding)

model_dropout = TimeDistributed(Dropout(dropout_ratio))(model_bilstm)

model_dense = TimeDistributed(Dense(tag_size, activation='relu'))(model_dropout)

base = Model(inputs=sequence_input, outputs=model_dense)
model = CRFModel(base, tag_size)

In [16]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/AURA2/bilstm_crf_all_ner/cp.ckpt')

In [17]:
model.summary()

Model: "crf_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence_input (InputLayer)    [(None, 40)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 40, 64)       14871232    ['sequence_input[0][0]']         
                                                                                                  
 bidirectional (Bidirectional)  (None, 40, 64)       24832       ['embedding[0][0]']              
                                                                                                  
 time_distributed (TimeDistribu  (None, 40, 64)      0           ['bidirectional[0][0]']          
 ted)                                                                                     

In [50]:
def apply_ner(talkid_tokens, id='MDRW2100000037.62') :
  max_len = 40
  X_data = src_tokenizer.texts_to_sequences(talkid_tokens[id])
  X_data = pad_sequences(X_data, maxlen=max_len, padding = 'post')
  y = model.predict(X_data)[0]
  for i in range(len(X_data)) :
    if i % 2 == 0 : # speaker1
      sent = talkid_tokens[id][i]
      ner = [index_to_ner[t] for t in y[i]]
      print("Speaker1 : ", sent)
      print("NER1 : ", ner)
    elif i % 2 == 1 : # speaker2
      sent = talkid_tokens[id][i]
      ner = [index_to_ner[t] for t in y[i]]
      print("Speaker2 : ", sent)
      print("NER2 : ", ner)
  
  

In [51]:
apply_ner(talkid_tokens)

Speaker1 :  ['니', '미역국', '먹을래']
NER1 :  ['O', 'B-CV_FOOD', 'O', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Speaker2 :  ['왜', '갑자기']
NER2 :  ['O', 'O', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Speaker1 :  ['아니', '우리', '집에', '아빠', '친구', '가', '주신', '미역', '이', '많으니', '까', '부지런히', '먹으라고']
NER1 :  ['O', 'O', 'I-AFA_VIDEO', 'I-AFA_VIDEO', 'I-AFA_VIDEO', 'O', 'O', 'B-CV_FOOD', 'O', 'O', 'O', 'O', 'O', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', '

In [53]:
cnt = 0
for i in talkid_tokens :
  if cnt == 10 :
    break
  apply_ner(talkid_tokens, id=i)
  cnt += 1
   

Speaker1 :  ['안녕', '하세요']
NER1 :  ['O', 'O', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Speaker2 :  ['️']
NER2 :  ['O', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Speaker1 :  ['이거', '해봐요><']
NER1 :  ['O', 'O', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Speaker2 :  ['나의', '직장', '인', '멘탈', '성향', '은', '[안챙겨도', '잘커요', '탕비실', '